<a href="https://colab.research.google.com/github/OskarMyhreMidboe/DAT255-Chestxray/blob/main/DAT255_semesteroppgave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

In [49]:
#!pip -q install torchxrayvision

In [50]:
from fastbook import *

import torchxrayvision as xrv
import matplotlib.pyplot as plt
import pandas as pd

In [51]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

fatal: destination path 'covid-chestxray-dataset' already exists and is not an empty directory.


In [55]:
import csv

my_file_name = "/content/covid-chestxray-dataset/metadata.csv"
cleaned_file = "/content/covid-chestxray-dataset/cleanmetadata.csv"
remove_words = ['.nii.gz']

with open(my_file_name, 'r', newline='') as infile, \
     open(cleaned_file, 'w',newline='') as outfile:
    writer = csv.writer(outfile)
    for line in csv.reader(infile, delimiter='|'):
        if not any(remove_word in line for remove_word in remove_words):
            writer.writerow(line)

In [58]:
d = xrv.datasets.COVID19_Dataset(imgpath="covid-chestxray-dataset/images/",csvpath="covid-chestxray-dataset/cleanmetadata.csv")

KeyError: ignored

In [ ]:
sample = d[1]
sample2 = d[2]

In [ ]:
plt.imshow(sample["img"][0], cmap="gray");


In [ ]:
plt.imshow(sample2["img"][0], cmap="gray");

In [ ]:
data_df = pd.read_csv(d.csvpath)

In [ ]:
data_df.head()
#Trenger : patientid, sex, age, finding, folder, img_id

In [ ]:
data_df.shape

In [ ]:
#Lager ny dataframe med utvalgte 

final_df = data_df.filter(['patientid', 'sex', 'age', 'finding', 'filename','folder' ], axis=1)

In [ ]:
final_df.head()

In [ ]:
final_df['ctgry'] = "other"
final_df.loc[final_df.finding == "Pneumonia/Viral/COVID-19", 'ctgry'] = "covid"


In [ ]:
df_ex = final_df.filter(['filename', 'ctgry'], axis=1)

In [ ]:
final_df

In [ ]:
from fastai.vision.all import *

In [ ]:
df_ex.head()

In [ ]:
data = DataBlock(blocks = (ImageBlock(cls=PILImageBW), CategoryBlock),
                  get_x = lambda x:str(d.imgpath)+f"{x[0]}",
                  splitter = RandomSplitter(),
                  get_y = lambda x: if x[1],
                  batch_tfms = [Normalize.from_stats(*imagenet_stats)],
                  item_tfms = Resize(256)
                 )

In [ ]:
dls = data.dataloaders(df_ex.values, bs=20)

In [ ]:
np.unique(final_df.finding)

In [ ]:
d.imgpath

In [ ]:
dls.show_batch()

In [ ]:
#covid-chestxray-dataset/images/coronacases_org_002.nii.gz
#covid-chestxray-dataset/images/coronacases_org_001.nii.gz

In [59]:
learn = cnn_learner(dls, models.resnet34, metrics=[accuracy])

learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time


FileNotFoundError: ignored